In [1]:
print("Hello World !!")

Hello World !!


In [2]:
%pwd

'd:\\Medical Chatbot GENAI\\End-to-End-Medical-Chatbot-GenAI\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'd:\\Medical Chatbot GENAI\\End-to-End-Medical-Chatbot-GenAI'

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


In [6]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='data/')

In [8]:
#extracted_data 

In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts = text_splitter.split_documents(extracted_data)

    return texts

In [10]:
text_chunks=text_split(extracted_data=extracted_data)
print(f"Total number of text chunks: {len(text_chunks)}")

Total number of text chunks: 5859


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings


In [12]:
## Download embeedings from Hugging Face
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
#!pip install sentence-transformers

In [14]:
# from sentence_transformers import SentenceTransformer


In [15]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("all-MiniLM-L6-v2")
# print("Success ✅")


In [16]:
embeedings=download_huggingface_embeddings()

c:\Users\HP\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# from langchain.vectorstores import FAISS

# vectorstore = FAISS.from_documents(extracted_data, embeedings)
# retriever = vectorstore.as_retriever()


In [18]:
query_result=embeedings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [19]:
query_result 

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [65]:
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY= os.getenv("GEMINI_API_KEY")
GROQ_API_KEY= os.getenv("GROQ_API_KEY")

In [26]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name ="medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '2fada3c1f337d63e2fa557173bb72735', 'date': 'Sat, 28 Jun 2025 06:33:35 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [27]:
import os 
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY


In [67]:
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [29]:
#Embed each chunk and upsert the embeedings into ur pinecone 
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeedings,
    index_name=index_name
)


In [30]:
## Load Existinf Index

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeedings
)

In [31]:
docsearch 

In [32]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [33]:
retrieved_docs = retriever.invoke("What is Acne?")

In [34]:
retrieved_docs

[Document(id='0daa3c6d-69dc-4f65-a236-9abf122bc2d4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8406b476-174a-400c-a783-aea163606703', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='7c1879fa-f930-4f60-aee3-360d2ecdafba', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 

In [68]:
# !pip install langchain-google-genai google-generativeai



In [56]:
!pip install langchain-groq



   ---------------------------------------- 0/2 [groq]
   -------------------- ------------------- 1/2 [langchain-groq]
   ---------------------------------------- 2/2 [langchain-groq]



In [71]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama3-8b-8192",  # or "llama3-70b-8192", "mixtral-8x7b-32768"
    groq_api_key=GROQ_API_KEY,  
    temperature=0.6,
    max_tokens=300
)

# response = llm.invoke("What is Acne?")
# print(response)



In [72]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt =(
    "You are an assistant for a question answering system. "
    "Use the following pieces of context to answer the question. "
    "the question . If you don't know the answer, just say that you don't know, "
    "Use five sentences maximum to answer the question. "
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [73]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)


In [61]:
# retriever = vectorstore.as_retriever()


In [74]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)


In [75]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response['answer'])

Acne is a skin disorder in which the sebaceous glands become inflamed. It is characterized by the formation of comedones (blackheads and whiteheads), papules, pustules, nodules, and cysts. Acne can occur on the face, back, chest, and other areas of the body, and can cause scarring and other complications if left untreated.


In [ ]:
# # !pip install --upgrade langchain-google-genai google-generativeai langchain
# !pip install -q -U google-genai


In [76]:
response = rag_chain.invoke({"input": "What is Stats?"})
print(response['answer'])

I don't know. The context provided doesn't mention "Stats". However, the context does describe a Complete Blood Count (CBC) which is a type of medical test that evaluates the cells in the blood.
